# Test alpha-spheres network 2

In [1]:
import molsysmt as msm

In [2]:
from scipy.spatial import Voronoi
from math import sqrt

In [3]:
from scipy.spatial.distance import euclidean

In [4]:
import numpy as np

In [5]:
from tqdm import tqdm

In [6]:
molecular_system = msm.convert('1HIV')
molecular_system = msm.extract(molecular_system, selection='molecule_type in ["protein", "peptide"]')

In [7]:
msm.info(molecular_system)

form,n_atoms,n_groups,n_components,n_chains,n_molecules,n_entities,n_proteins,n_structures
molsysmt.MolSys,1516,198,2,2,2,1,2,1


## Castp

In [8]:
pocket = []
with open('castp/1hiv/1hiv.poc') as file:
    for line in file:
        fields = line.rstrip().split()
        if fields[-2]=='1':
            pocket.append(int(fields[1]))

pocket = msm.get(molecular_system, element='atom', selection='atom_id==@pocket', atom_index=True)

In [9]:
view=msm.view(molecular_system)
view

NGLWidget()

In [10]:
view.clear()
#view.add_surface(color='w', opacity=0.5)
view.add_ball_and_stick(color='w', opacity=0.5)

In [11]:
sel='@'+','.join([str(ii) for ii in pocket])
#view.add_surface(sel, color='red', opacity=0.3)
view.add_ball_and_stick(sel, color='red', opacity=0.3)

In [12]:
pocket

array([  66,   67,   68,   69,   70,   71,  180,  181,  196,  197,  201,
        206,  207,  208,  210,  211,  213,  214,  218,  219,  220,  221,
        222,  225,  226,  229,  243,  365,  367,  368,  369,  370,  371,
        372,  373,  374,  375,  376,  377,  378,  381,  382,  383,  401,
        402,  403,  585,  610,  616,  617,  618,  624,  625,  640,  641,
        661,  825,  826,  827,  828,  829,  830,  939,  940,  956,  960,
        965,  966,  967,  969,  970,  972,  973,  977,  978,  979,  980,
        981,  984,  985,  988, 1002, 1121, 1124, 1126, 1127, 1128, 1129,
       1130, 1131, 1132, 1133, 1136, 1140, 1141, 1143, 1161, 1162, 1370,
       1376, 1377, 1383, 1399, 1400, 1420])

# Mine

In [93]:
radio = {'C': 1.7, 'H':1.1, 'O':1.52, 'N':1.55, 'P':1.8, 'S':1.8}

In [13]:
coordinates = msm.get(molecular_system, coordinates=True)

In [14]:
coordinates = msm.pyunitwizard.get_value(coordinates[0])
n_atoms = coordinates.shape[0]

In [94]:
atom_types = msm.get(molecular_system, element='atom', atom_type=True)
radii = np.array([radio[ii]*0.1 for ii in atom_types])
probe = 0.1

In [16]:
voronoi = Voronoi(coordinates)

In [17]:
centers = voronoi.vertices
n_alphaspheres = centers.shape[0]

In [18]:
atoms_of_alphasphere = {ii:[] for ii in range(n_alphaspheres)}
atoms_of_alphasphere[-1] = []
alphaspheres_of_atom = {ii:[] for ii in range(n_atoms)}

In [19]:
for atom_index, region_index in enumerate(voronoi.point_region):
    for alphasphere_index in voronoi.regions[region_index]:
        atoms_of_alphasphere[alphasphere_index].append(atom_index)
        alphaspheres_of_atom[atom_index].append(alphasphere_index)

In [20]:
alphaspheres_in_castp = []
for ii in range(n_alphaspheres):
    if np.isin(atoms_of_alphasphere[ii], pocket).all():
        alphaspheres_in_castp.append(ii)

In [121]:
alphaspheres_in_castp

[385,
 386,
 391,
 394,
 686,
 869,
 1797,
 1803,
 1804,
 1805,
 1820,
 1823,
 2487,
 2731,
 2735,
 2737,
 2879,
 2882,
 2886,
 2971,
 2972,
 2973,
 2974,
 3054,
 3060,
 3061,
 3929,
 4167,
 4526,
 4530,
 4651,
 4652,
 4654,
 4660,
 4662,
 4666,
 4667,
 4668,
 4944,
 5070,
 5071,
 5076,
 6618,
 6619,
 6722,
 6723,
 6744,
 6750,
 6755,
 6763,
 6764,
 6832,
 6835,
 6836,
 6837,
 6842,
 6843,
 7127,
 7129,
 7130,
 7131,
 7132,
 7133,
 7134,
 7137,
 7138,
 7139,
 7140,
 7953,
 7991,
 7992,
 7994,
 7995,
 7996,
 7997,
 8005,
 8006,
 8007,
 8008,
 8009,
 8011,
 8230,
 8303,
 8305,
 8306,
 8307,
 8312,
 8313,
 8314,
 8315,
 8316,
 8317,
 8445,
 8467,
 8471,
 8472,
 8487,
 8503,
 8506,
 8507,
 8508,
 8509,
 8510,
 8513,
 8514,
 8515,
 8516,
 8517,
 8520,
 8521,
 8522,
 8523,
 8524,
 8526,
 8527,
 8528,
 8529,
 8535,
 8536,
 8537,
 8601,
 8602,
 8604,
 8718,
 8721,
 8722,
 8723,
 8724,
 8725,
 8726,
 8727,
 8728,
 8729,
 8730,
 8731,
 8732,
 9210,
 9244,
 9249,
 9250,
 9255,
 9256,
 9259,
 9260

In [123]:
alphaspheres_in_castp[-1]

9548

In [124]:
atoms_of_alphasphere[9548]

[610, 617, 618, 1140]

In [125]:
pocket

array([  66,   67,   68,   69,   70,   71,  180,  181,  196,  197,  201,
        206,  207,  208,  210,  211,  213,  214,  218,  219,  220,  221,
        222,  225,  226,  229,  243,  365,  367,  368,  369,  370,  371,
        372,  373,  374,  375,  376,  377,  378,  381,  382,  383,  401,
        402,  403,  585,  610,  616,  617,  618,  624,  625,  640,  641,
        661,  825,  826,  827,  828,  829,  830,  939,  940,  956,  960,
        965,  966,  967,  969,  970,  972,  973,  977,  978,  979,  980,
        981,  984,  985,  988, 1002, 1121, 1124, 1126, 1127, 1128, 1129,
       1130, 1131, 1132, 1133, 1136, 1140, 1141, 1143, 1161, 1162, 1370,
       1376, 1377, 1383, 1399, 1400, 1420])

In [24]:
edges=[]
interfaces={}

for ii in tqdm(range(n_alphaspheres)):
    
    neighbors=[]
    for jj in atoms_of_alphasphere[ii]:
        for kk in alphaspheres_of_atom[jj]:
            if kk>ii:
                neighbors.append(kk)
    neighbors=np.unique(neighbors)

    interfaces[ii]={}
    
    for jj in neighbors:
        common = np.intersect1d(atoms_of_alphasphere[ii], atoms_of_alphasphere[jj], assume_unique=True)
        if len(common)>=3:
            edges.append([ii,jj])
            interfaces[ii][jj]=common

100%|█████████████████████████████████████████████████████████| 9810/9810 [00:04<00:00, 2032.07it/s]


In [25]:
len(edges)

19547

In [26]:
import networkx as nx

In [27]:
G = nx.Graph(edges)

In [126]:
G[9548]

AtlasView({9208: {}, 9544: {}, 9551: {}, 9552: {}})

In [127]:
interfaces[9548]

{9551: array([ 617,  618, 1140]), 9552: array([ 610,  618, 1140])}

In [155]:
aaa = 9548
bbb = 9552

In [156]:
face_coordinates = coordinates[interfaces[aaa][bbb]]

In [157]:
centroid = face_coordinates.mean(axis=1)

In [158]:
euclidean(centroid, face_coordinates[0])

1.3585085073139749

In [159]:
euclidean(centroid, face_coordinates[1])

1.4311952630969371

In [160]:
euclidean(centroid, face_coordinates[2])

1.074609706410244

In [161]:
view=msm.view(molecular_system)

view.clear()
#view.add_surface(color='w', opacity=0.5)
view.add_ball_and_stick(color='w', opacity=0.5)

for ii in [aaa, bbb]:
    aa=atoms_of_alphasphere[ii]
    view.shape.add_cylinder(coordinates[aa[0]]*10, coordinates[aa[1]]*10, [1,0,0], 0.1)
    view.shape.add_cylinder(coordinates[aa[0]]*10, coordinates[aa[2]]*10, [1,0,0], 0.1)
    view.shape.add_cylinder(coordinates[aa[0]]*10, coordinates[aa[3]]*10, [1,0,0], 0.1)
    view.shape.add_cylinder(coordinates[aa[1]]*10, coordinates[aa[2]]*10, [1,0,0], 0.1)
    view.shape.add_cylinder(coordinates[aa[1]]*10, coordinates[aa[3]]*10, [1,0,0], 0.1)
    view.shape.add_cylinder(coordinates[aa[2]]*10, coordinates[aa[3]]*10, [1,0,0], 0.1)

view

NGLWidget()

In [162]:
def face_is_permeable(atom_indices, coordinates, radii, probe_radius):

    if len(atom_indices)>3:
        raise ValueError('no se ver si es permeable')
    
    c = coordinates[atom_indices]
    r = radii[atom_indices]
    rp = probe_radius
    
    pasa=False

    for ii,jj,kk in [[0, 1, 2], [0, 2, 1], [1, 2, 0]]:
        
        v21=c[ii]-c[jj]
        d21=sqrt(v21[0]**2+v21[1]**2+v21[2]**2)
        u21=v21/d21

        if d21>=(r[ii]+r[jj]+2*rp):

            t=c[ii]+(r[ii]+rp)*u21
            pasa = ((t[0]-c[kk][0])**2+(t[1]-c[kk][1])**2+(t[2]-c[kk][2])**2)>=(r[kk]+rp)**2
        
            if not pasa:
                t=c[jj]-(r[jj]+rp)*u21
                pasa = ((t[0]-c[kk][0])**2+(t[1]-c[kk][1])**2+(t[2]-c[kk][2])**2)>=(r[kk]+rp)**2

        if pasa:
            break
    
    return pasa

In [163]:
face_is_permeable(interfaces[aaa][bbb], coordinates, radii, probe)

False

In [111]:
interfaces[aaa][bbb]

array([ 616, 1130, 1161])

In [ ]:
view=msm.view(molecular_system)
view

In [ ]:
view.clear()
#view.add_surface(color='w', opacity=0.5)
view.add_ball_and_stick(color='w', opacity=0.5)

In [ ]:
for ii in range(len(pockets)):
    if len(pockets[ii])>10:
        print(ii, len(pockets[ii]))

In [ ]:
pocket = 40
alphaspheres_in_pocket = pockets[pocket]
atoms_in_pocket = np.concatenate([atoms_of_alphasphere[ii] for ii in alphaspheres_in_pocket])
atoms_in_pocket = np.unique(atoms_in_pocket)

print(alphaspheres_in_pocket)

In [ ]:
sel='@'+','.join([str(ii) for ii in atoms_in_pocket])
view.add_surface(sel, color='red', opacity=0.3)

In [ ]:
for ii in alphaspheres_in_pocket:
    aa=atoms_of_alphasphere[ii]
    view.shape.add_cylinder(coordinates[aa[0]]*10, coordinates[aa[1]]*10, [1,0,0], 0.1)
    view.shape.add_cylinder(coordinates[aa[0]]*10, coordinates[aa[2]]*10, [1,0,0], 0.1)
    view.shape.add_cylinder(coordinates[aa[0]]*10, coordinates[aa[3]]*10, [1,0,0], 0.1)
    view.shape.add_cylinder(coordinates[aa[1]]*10, coordinates[aa[2]]*10, [1,0,0], 0.1)
    view.shape.add_cylinder(coordinates[aa[1]]*10, coordinates[aa[3]]*10, [1,0,0], 0.1)
    view.shape.add_cylinder(coordinates[aa[2]]*10, coordinates[aa[3]]*10, [1,0,0], 0.1)